In [84]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import compute_class_weight
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [63]:
data = pd.read_csv('/kaggle/input/bank-marketing-campaigns-dataset/bank-additional-full.csv',delimiter=';')

### https://www.kaggle.com/datasets/volodymyrgavrysh/bank-marketing-campaigns-dataset/code

In [64]:
data

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,...,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [65]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

this is a training but i think maybe onehot encoding for all of this is better then encoding one by one

because we have missing value in binary columns (unknown)

for check this behind line 10 enter colunms

In [66]:
y = data['y'].copy()

In [67]:
data = data.drop('y',axis=1)

In [68]:
columns = {col : data[col].unique() for col in data.columns if data[col].dtypes =='object'}
for col in columns :
    dummie = pd.get_dummies(data[col],prefix=col[0:3]).astype(np.int)
    data = pd.concat([data,dummie],axis=1)
    data = data.drop(col,axis=1)

/tmp/ipykernel_31/949819619.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dummie = pd.get_dummies(data[col],prefix=col[0:3]).astype(np.int)
/tmp/ipykernel_31/949819619.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
De

In [69]:
scaler = StandardScaler()
data = scaler.fit_transform(data)

In [70]:
y = y.apply(lambda x: 1 if x=='yes' else 0)

In [71]:
x_train,x_test,y_train,y_test=train_test_split(data,y,train_size=0.745,random_state=13)

In [75]:
weights = dict(enumerate(compute_class_weight(
    'balanced',
    classes = y.unique()
    ,y=y
)))

In [76]:
inputs=tf.keras.Input(shape = (data.shape[1]))
x = tf.keras.layers.Dense(64,activation='relu')(inputs)
x = tf.keras.layers.Dense(64,activation='relu')(x)

outputs = tf.keras.layers.Dense(1,activation='sigmoid')(x)

model = tf.keras.Model(inputs,outputs)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy'
    ,metrics=tf.keras.metrics.AUC(name='auc')
)

model.fit(
    x_train,
    y_train,
    validation_split =0.2,
    batch_size=32,
    class_weight=weights,
    epochs=100,
    callbacks=tf.keras.callbacks.EarlyStopping(
        monitor='val_auc'
        ,patience=3
        ,restore_best_weights=True
    )

)

Epoch 1/100
768/768 [==============================] - 3s 3ms/step - loss: 0.3667 - auc: 0.9175 - val_loss: 0.3173 - val_auc: 0.9329
Epoch 2/100
768/768 [==============================] - 2s 3ms/step - loss: 0.2924 - auc: 0.9417 - val_loss: 0.3332 - val_auc: 0.9373
Epoch 3/100
768/768 [==============================] - 2s 3ms/step - loss: 0.2758 - auc: 0.9469 - val_loss: 0.3109 - val_auc: 0.9365
Epoch 4/100
768/768 [==============================] - 2s 3ms/step - loss: 0.2661 - auc: 0.9496 - val_loss: 0.3187 - val_auc: 0.9360
Epoch 5/100
768/768 [==============================] - 2s 3ms/step - loss: 0.2576 - auc: 0.9520 - val_loss: 0.3065 - val_auc: 0.9361


In [77]:
model.evaluate(x_test,y_test)

329/329 [==============================] - 1s 2ms/step - loss: 0.3428 - auc: 0.9383


[0.34283724427223206, 0.9382621049880981]

In [82]:
y_true = np.array(y_test)
y_pred = np.squeeze(np.array(model.predict(x_test) >= 0.5,dtype=np.int))

329/329 [==============================] - 0s 1ms/step


/tmp/ipykernel_31/949034348.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_pred = np.squeeze(np.array(model.predict(x_test) >= 0.5,dtype=np.int))


In [83]:
print(confusion_matrix(y_true,y_pred))

[[7744 1570]
 [  84 1105]]


In [85]:
print('classification report \n\n\n ',classification_report(y_true,y_pred))

classification report 


                precision    recall  f1-score   support

           0       0.99      0.83      0.90      9314
           1       0.41      0.93      0.57      1189

    accuracy                           0.84     10503
   macro avg       0.70      0.88      0.74     10503
weighted avg       0.92      0.84      0.87     10503

